In [1]:
!pip install agentpy

In [4]:
import agentpy as ap
import numpy as np
import socket
import json
import time

class CircularAgent(ap.Agent):
    def setup(self):
        self.radius = self.p.radius
        self.speed = self.p.speed
        self.angle = 2 * np.pi * self.id / self.p.n
        self.x = self.radius * np.cos(self.angle)
        self.y = self.radius * np.sin(self.angle)
        print(f"Agent {self.id} initialized at position ({self.x:.2f}, {self.y:.2f})")

    def move(self):
        self.angle += self.speed
        self.x = self.radius * np.cos(self.angle)
        self.y = self.radius * np.sin(self.angle)
        
        data = {
            "id": f"agent_{self.id}",
            "position": {
                "x": float(self.x),
                "y": float(self.y)
            }
        }
        
        self.send_to_unity(data)
    
    def send_to_unity(self, data):
        message = json.dumps(data).encode('utf-8')
        MAX_RETRIES = 3
        
        for attempt in range(MAX_RETRIES):
            try:
                with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                    s.settimeout(5) 
                    s.connect(("127.0.0.1", 1114))
                    
                    initial_message = s.recv(1024)
                    print(f"Agent {self.id} received: {initial_message.decode('utf-8')}")
                    
                    s.sendall(message + b"$")
                    print(f"Agent {self.id} sent position: ({self.x:.2f}, {self.y:.2f})")
                    break 
                    
            except ConnectionRefusedError:
                print(f"Agent {self.id} attempt {attempt+1}: Unity server not running")
                if attempt == MAX_RETRIES - 1:
                    print(f"Agent {self.id}: Failed to connect after {MAX_RETRIES} attempts")
                else:
                    time.sleep(1)  
            except socket.timeout:
                print(f"Agent {self.id} attempt {attempt+1}: Connection timeout")
                if attempt < MAX_RETRIES - 1:
                    time.sleep(0.5)
            except Exception as e:
                print(f"Agent {self.id} attempt {attempt+1} failed: {e}")
                if attempt < MAX_RETRIES - 1:
                    time.sleep(0.5)

class CircleModel(ap.Model):
    def setup(self):
        print(f"Setting up model with {self.p.n} agents")
        self.agents = ap.AgentList(self, self.p.n, CircularAgent)
        print("Model setup complete")

    def step(self):
        print(f"Step {self.t}: Moving agents")
        for agent in self.agents:
            agent.move()
            time.sleep(self.p.agent_delay)  
        print(f"Step {self.t} complete")


parameters = {
    'n': 7,                
    'radius': 4.0,         
    'speed': 0.25,         
    'steps': 200,          
    'agent_delay': 0.2     
}

print("Starting circular agent simulation...")
print(f"Parameters: {parameters}")
print("Make sure Unity is running with the TCPIPServerAsync script!")
print("-" * 50)


model = CircleModel(parameters)
results = model.run()

print("-" * 50)
print("Simulation complete!")

Starting circular agent simulation...
Parameters: {'n': 7, 'radius': 4.0, 'speed': 0.25, 'steps': 200, 'agent_delay': 0.2}
Make sure Unity is running with the TCPIPServerAsync script!
--------------------------------------------------
Setting up model with 7 agents
Agent 1 initialized at position (2.49, 3.13)
Agent 2 initialized at position (-0.89, 3.90)
Agent 3 initialized at position (-3.60, 1.74)
Agent 4 initialized at position (-3.60, -1.74)
Agent 5 initialized at position (-0.89, -3.90)
Agent 6 initialized at position (2.49, -3.13)
Agent 7 initialized at position (4.00, -0.00)
Model setup complete
Step 1: Moving agents
Agent 1 received: I will send key
Agent 1 sent position: (1.64, 3.65)
Agent 2 received: I will send key
Agent 2 sent position: (-1.83, 3.56)
Agent 3 received: I will send key
Agent 3 sent position: (-3.92, 0.79)
Agent 4 received: I will send key
Agent 4 sent position: (-3.06, -2.57)
Agent 5 received: I will send key
Agent 5 sent position: (0.10, -4.00)
Agent 6 recei

KeyboardInterrupt: 